In [ ]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib.parse
import time
import numpy as np
import pandas as pd
import logging
import sqlite3
from datetime import datetime

# ロギング設定
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='scraping.log'
)

# ヘッダー設定
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# SUUMOを東京都品川区大井町のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?bs=040&ar=030&ta=13&sc=13109&oz=13109003&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1={}'

# データベース初期化関数
def init_database():
    conn = sqlite3.connect('suumo_properties.db')
    cursor = conn.cursor()
    
    # テーブル作成 (存在しない場合)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS properties (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        scrape_date TEXT,
        category TEXT,
        building_name TEXT,
        address TEXT,
        building_age TEXT,
        rent TEXT,
        url TEXT,
        page_number INTEGER
    )
    ''')
    
    conn.commit()
    return conn, cursor

# データベースへ挿入関数
def insert_to_database(conn, cursor, data_samples, current_page):
    scrape_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # データベースへの一括挿入
    insert_query = '''
    INSERT INTO properties (
        scrape_date, category, building_name, address, 
        building_age, rent, url, page_number
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    '''
    
    # データの準備
    insert_data = []
    for sample in data_samples:
        insert_data.append((
            scrape_date, 
            sample[0], sample[1], sample[2], 
            sample[3], sample[4], sample[5], 
            current_page
        ))
    
    # 一括挿入
    cursor.executemany(insert_query, insert_data)
    conn.commit()

    # プリント出力
    for sample in data_samples:
        print("---物件情報---")
        print(f"カテゴリ: {sample[0]}")
        print(f"建物名: {sample[1]}")
        print(f"住所: {sample[2]}")
        print(f"築年数: {sample[3]}")
        print(f"家賃: {sample[4]}")
        print(f"URL: {sample[5]}")
        print("---------------\n")

# ページ数の自動検出関数
def get_total_pages(soup):
    try:
        page_links = soup.find('div', class_='pagination-parts').find_all('a')
        return int(page_links[-2].text)  # 最後から2番目が最大ページ数
    except:
        return 2000  # デフォルト値

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    try:
        html = requests.get(url, headers=headers, timeout=10)
        html.raise_for_status()
        soup = BeautifulSoup(html.content, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        logging.error(f"ページ読み込みエラー: {e}")
        raise

# CSV出力関数
def export_to_csv(conn):
    # データベースからデータを取得
    df = pd.read_sql_query("SELECT * FROM properties", conn)
    
    # CSV出力
    filename = f'suumo_properties_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\nCSVファイル {filename} に出力しました。")
    
    # データ概要を表示
    print("\n--- データ概要 ---")
    print(f"総物件数: {len(df)}")
    
    # 家賃の平均計算（数値に変換可能な場合のみ）
    try:
        df['rent_numeric'] = df['rent'].str.replace('万円', '').astype(float)
        print(f"平均家賃: {df['rent_numeric'].mean():.2f}万円")
    except:
        print("家賃の平均計算に失敗しました。")
    
    print(f"物件カテゴリ内訳:\n{df['category'].value_counts()}")

def main():
    # データベース初期化
    conn, cursor = init_database()
    
    # 複数ページの情報をまとめて取得
    data_samples = []

    # 最初のページで総ページ数を取得
    first_page_soup = load_page(url.format(1))
    max_page = get_total_pages(first_page_soup)
    
    # 処理時間を測りたい
    start = time.time()
    times = []

    # 均一な待機時間のための基準時間
    base_start_time = time.time()

    try:
        # ページごとの処理
        for page in range(1, max_page+1):
            # 均一な待機時間の実装
            ideal_elapsed_time = (page - 1) * 1.0  # ページごとに1秒間隔
            actual_elapsed_time = time.time() - base_start_time
            
            # 理想の経過時間より早く処理が進んでいる場合、待機
            if actual_elapsed_time < ideal_elapsed_time:
                time.sleep(ideal_elapsed_time - actual_elapsed_time)
            
            before = time.time()
            
            # ページ情報
            soup = load_page(url.format(page))
            
            # 物件情報リストを指定
            mother = soup.find_all(class_='cassetteitem')
            
            # 物件ごとの処理
            for child in mother:
                # 建物情報
                data_home = []
                
                # カテゴリ
                try:
                    data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
                except:
                    data_home.append('不明')
                
                # 建物名
                try:
                    data_home.append(child.find(class_='cassetteitem_content-title').text)
                except:
                    data_home.append('不明')
                
                # 住所
                try:
                    data_home.append(child.find(class_='cassetteitem_detail-col1').text)
                except:
                    data_home.append('不明')
                
                # 築年数
                try:
                    children = child.find(class_='cassetteitem_detail-col3')
                    data_home.append(children.find_all('div')[0].text)  # 築年数
                except:
                    data_home.append('不明')

                # 家賃
                try:
                    room = child.find(class_='cassetteitem_other')
                    rent = room.find(class_='cassetteitem_other-emphasis ui-text--bold').text
                    data_home.append(rent)
                except:
                    data_home.append('不明')
                
                # URL
                try:
                    get_url = child.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url, get_url)
                    data_home.append(abs_url)
                except:
                    data_home.append('不明')

                # データを保存
                data_samples.append(data_home)
            
            # データベースに保存
            insert_to_database(conn, cursor, data_samples, page)
            
            # データサンプルをリセット
            data_samples = []
            
            # 進捗確認
            after = time.time()
            running_time = after - before
            times.append(running_time)
            
            print(f'{page}ページ目：{running_time}秒')
            
            # 作業進捗
            complete_ratio = round(page/max_page*100, 3)
            print(f'完了：{complete_ratio}%')
            
            # 作業の残り時間目安を表示
            running_mean = np.mean(times)
            running_required_time = running_mean * (max_page - page)
            hour = int(running_required_time/3600)
            minute = int((running_required_time%3600)/60)
            second = int(running_required_time%60)
            print(f'残り時間：{hour}時間{minute}分{second}秒\n')

            # ログ記録
            logging.info(f'ページ {page} スクレイピング完了')

    except Exception as e:
        logging.error(f"スクレイピング中にエラー発生: {e}")
        print(f"エラー: {e}")
    
    # 処理時間を測りたい
    finish = time.time()
    running_all = finish - start
    print('総経過時間：', running_all)

    # CSV出力
    export_to_csv(conn)

    # データベース接続を閉じる
    conn.close()
    logging.info('データベース接続終了')

if __name__ == "__main__":
    main()


---物件情報---
カテゴリ: 賃貸アパート
建物名: ＪＲ山手線 大崎駅 2階建 築58年
住所: 東京都品川区大崎３
築年数: 築58年
家賃: 8万円
URL: https://suumo.jp/chintai/jnc_000093530231/?bc=100415100557
---------------

---物件情報---
カテゴリ: 賃貸アパート
建物名: 森美荘
住所: 東京都品川区大崎３
築年数: 築58年
家賃: 8万円
URL: https://suumo.jp/chintai/jnc_000093530230/?bc=100404241744
---------------

---物件情報---
カテゴリ: 賃貸マンション
建物名: 大島マンション
住所: 東京都品川区大崎４
築年数: 築52年
家賃: 5.5万円
URL: https://suumo.jp/chintai/jnc_000051575227/?bc=100390336824
---------------

---物件情報---
カテゴリ: 賃貸アパート
建物名: シャンテ大崎
住所: 東京都品川区大崎３
築年数: 築52年
家賃: 7万円
URL: https://suumo.jp/chintai/jnc_000077353217/?bc=100413239857
---------------

---物件情報---
カテゴリ: 賃貸アパート
建物名: ＪＲ山手線 大崎駅 2階建 築52年
住所: 東京都品川区大崎３
築年数: 築52年
家賃: 7万円
URL: https://suumo.jp/chintai/jnc_000077643528/?bc=100413240571
---------------

---物件情報---
カテゴリ: 賃貸マンション
建物名: ＪＲ山手線 大崎駅 5階建 築47年
住所: 東京都品川区大崎２
築年数: 築47年
家賃: 6.8万円
URL: https://suumo.jp/chintai/jnc_000093175213/?bc=100399056729
---------------

---物件情報---
カテゴリ: 賃貸マンション
建物名: 若松コーポラス
住所: 東京都品川区大崎２
築年数: 築50年
家賃: 

In [8]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np

# データベース接続
conn = sqlite3.connect('suumo_properties.db')

# データを取得するSQLクエリ
query = '''
SELECT building_age, rent
FROM properties
WHERE building_age != '不明' AND rent != '不明'
'''

# データの取得
df = pd.read_sql_query(query, conn)

# 家賃と築年数のデータクリーニング
# 築年数を数値型に変換
df['building_age'] = pd.to_numeric(df['building_age'], errors='coerce')

# 家賃を数値型に変換（万円単位）
df['rent'] = df['rent'].str.replace('万円', '').astype(float)

# 不要な欠損値を除去
df.dropna(subset=['building_age', 'rent'], inplace=True)

# 相関関係の分析
correlation = df['building_age'].corr(df['rent'])
print(f"築年数と家賃の相関関係: {correlation:.2f}")

# データの可視化
plt.figure(figsize=(10, 6))
sns.scatterplot(x='building_age', y='rent', data=df)
plt.title("築年数と家賃の関係")
plt.xlabel("築年数")
plt.ylabel("家賃 (万円)")
plt.show()

# 線形回帰による分析
X = df[['building_age']]
y = df['rent']

# 回帰モデルを作成
model = LinearRegression()
model.fit(X, y)

# 回帰直線のプロット
plt.figure(figsize=(10, 6))
sns.regplot(x='building_age', y='rent', data=df, line_kws={'color': 'red'})
plt.title("築年数と家賃の回帰分析")
plt.xlabel("築年数")
plt.ylabel("家賃 (万円)")
plt.show()

# 回帰モデルの係数と切片
print(f"回帰係数: {model.coef_[0]:.2f}")
print(f"切片: {model.intercept_:.2f}")

# 仮説検証の結果
if correlation < 0:
    print("仮説：築年数が増えると家賃は低くなる、が支持されました。")
else:
    print("仮説：築年数が増えると家賃は低くなる、が支持されませんでした。")

# データベース接続を閉じる
conn.close()


ModuleNotFoundError: No module named 'matplotlib'